<a href="https://colab.research.google.com/github/alxmarqs/LLMtopics/blob/main/4_exercicio_comparacao_atencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício: Comparação Encoder-Decoder COM e SEM Atenção (Luong)

Este notebook compara dois modelos Seq2Seq baseados em GRU para a tarefa de **inverter sequências de caracteres** (ex: `abcd` → `dcba`):

- **Modelo COM atenção**: usa Luong Attention (dot product)
- **Modelo SEM atenção**: decoder simples sem mecanismo de atenção

Ao final, comparamos a perda (loss) ao longo do treinamento e a acurácia por sequência.

## 1. Preparação dos dados

In [ ]:
import torch
import torch.nn as nn
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.manual_seed(42)
random.seed(42)

chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)}
inv_vocab = {i: ch for ch, i in vocab.items()}
vocab_size = len(vocab)

def encode(s):
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t):
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5):
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados (mesmos dados para ambos os modelos)
seqs = [random_seq() for _ in range(50000)]
pairs = [(encode(s), encode(s[::-1])) for s in seqs]

max_len = max(len(x) for x, _ in pairs)

def pad(x):
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs  = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Dispositivo: {device}')
print(f'Exemplo de par - entrada: {decode(pairs[0][0])}  |  saída: {decode(pairs[0][1])}')

## 2. Definição dos modelos

### 2.1 Encoder (compartilhado)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru   = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        outputs, h = self.gru(x)
        return outputs, h   # outputs: (B, S, H)  |  h: (1, B, H)

### 2.2 Modelo COM atenção de Luong (dot-product)

In [ ]:
class LuongAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden  : (B, 1, H)
        encoder_outputs : (B, S, H)
        """
        # score = h_t · h_s^T  →  (B, 1, S)
        attn_scores  = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2))
        attn_weights = F.softmax(attn_scores, dim=-1)
        # context = soma ponderada  →  (B, 1, H)
        context = torch.bmm(attn_weights, encoder_outputs)
        return context, attn_weights


class DecoderComAtencao(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru   = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.attn  = LuongAttention()
        # [h_t ; context]  →  vocab
        self.fc    = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x, h, encoder_outputs):
        x = self.embed(x)          # (B, T, E)
        outputs = []
        hidden  = h
        for t in range(x.size(1)):
            inp = x[:, t:t+1]                              # (B, 1, E)
            out_t, hidden = self.gru(inp, hidden)          # (B, 1, H)
            context, _    = self.attn(out_t, encoder_outputs)
            combined      = torch.cat([out_t, context], dim=-1)
            outputs.append(self.fc(combined))
        return torch.cat(outputs, dim=1), hidden           # (B, T, V)

### 2.3 Modelo SEM atenção

In [ ]:
class DecoderSemAtencao(nn.Module):
    """
    Decoder padrão sem mecanismo de atenção.
    Usa apenas o estado oculto do GRU para gerar a saída.
    O contexto é simplesmente ignorado — toda a informação da
    sequência de entrada precisa estar comprimida no vetor h.
    """
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru   = nn.GRU(emb_size, hidden_size, batch_first=True)
        # apenas h_t  →  vocab  (sem concatenar contexto)
        self.fc    = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h, encoder_outputs=None):
        # encoder_outputs aceito mas ignorado para manter interface igual
        x = self.embed(x)          # (B, T, E)
        outputs = []
        hidden  = h
        for t in range(x.size(1)):
            inp = x[:, t:t+1]                        # (B, 1, E)
            out_t, hidden = self.gru(inp, hidden)    # (B, 1, H)
            outputs.append(self.fc(out_t))           # apenas h_t
        return torch.cat(outputs, dim=1), hidden     # (B, T, V)

### 2.4 Módulo Seq2Seq genérico

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        encoder_outputs, h = self.encoder(src)
        logits, _          = self.decoder(tgt[:, :-1], h, encoder_outputs)
        return logits

## 3. Funções de inferência

In [ ]:
def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        encoder_outputs, h = model.encoder(src)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)
        result = []
        for _ in range(max_len):
            logits, h = model.decoder(token, h, encoder_outputs)
            token = logits[:, -1, :].argmax(-1, keepdim=True)
            result.append(token.item())
        return decode(result)

## 4. Treinamento

Ambos os modelos usam os mesmos hiperparâmetros e dados para uma comparação justa.

In [ ]:
EMB_SIZE    = 32
HIDDEN_SIZE = 64
EPOCHS      = 10
LR          = 1e-3

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' '])

def build_model(decoder_cls):
    enc = Encoder(vocab_size, EMB_SIZE, HIDDEN_SIZE)
    dec = decoder_cls(vocab_size, EMB_SIZE, HIDDEN_SIZE)
    return Seq2Seq(enc, dec).to(device)

def train(model, label):
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    history = []
    for epoch in range(EPOCHS):
        model.train()
        total = 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
            opt.zero_grad()
            logits = model(xb, yb)
            loss   = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
            loss.backward()
            opt.step()
            total += loss.item()
        avg = total / len(train_dl)
        history.append(avg)
        print(f"[{label}] Epoch {epoch+1:2d}: loss = {avg:.4f}")
    return history

print("=" * 50)
print("Treinando modelo COM atenção de Luong...")
print("=" * 50)
model_com = build_model(DecoderComAtencao)
hist_com  = train(model_com, "COM atenção")

print()
print("=" * 50)
print("Treinando modelo SEM atenção...")
print("=" * 50)
model_sem = build_model(DecoderSemAtencao)
hist_sem  = train(model_sem, "SEM atenção")

## 5. Comparação da Loss durante o treinamento

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(range(1, EPOCHS+1), hist_com, marker='o', label='COM atenção (Luong)', color='steelblue')
plt.plot(range(1, EPOCHS+1), hist_sem, marker='s', label='SEM atenção',         color='tomato',    linestyle='--')
plt.xlabel('Época')
plt.ylabel('Cross-Entropy Loss')
plt.title('Curva de Loss: COM vs SEM Atenção')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 6. Comparação qualitativa: predições nos mesmos exemplos

In [ ]:
print(f"{'Entrada':<10} {'Esperado':<10} {'COM atenção':<15} {'SEM atenção':<15} {'Correto (COM)':<15} {'Correto (SEM)'}")
print("-" * 85)

acertos_com = 0
acertos_sem = 0
N_TEST = 20

for _ in range(N_TEST):
    s        = random_seq()
    esperado = s[::-1]
    pred_com = predict(model_com, s, max_len=len(s))
    pred_sem = predict(model_sem, s, max_len=len(s))
    ok_com   = '✓' if pred_com == esperado else '✗'
    ok_sem   = '✓' if pred_sem == esperado else '✗'
    acertos_com += pred_com == esperado
    acertos_sem += pred_sem == esperado
    print(f"{s:<10} {esperado:<10} {pred_com:<15} {pred_sem:<15} {ok_com:<15} {ok_sem}")

print("-" * 85)
print(f"Acurácia  →  COM atenção: {acertos_com}/{N_TEST} ({100*acertos_com/N_TEST:.0f}%)   |   SEM atenção: {acertos_sem}/{N_TEST} ({100*acertos_sem/N_TEST:.0f}%)")

## 7. Conclusões

| Aspecto | COM Atenção (Luong) | SEM Atenção |
|---|---|---|
| **Mecanismo** | O decoder consulta *todos* os estados do encoder a cada passo, ponderando qual posição de entrada é mais relevante | O decoder depende apenas do vetor `h` final do encoder, que deve comprimir toda a sequência |
| **Loss esperada** | Menor — o modelo aprende a alinhar posições de saída com as de entrada | Maior — o "gargalo" do vetor `h` dificulta aprender o alinhamento |
| **Acurácia esperada** | Mais alta | Mais baixa |
| **Interpretabilidade** | Os pesos de atenção revelam *qual* parte da entrada o modelo foca em cada passo de saída | Caixa-preta comprimida em `h` |
| **Custo computacional** | Levemente maior (produto escalar + softmax por passo) | Menor |

### Por que a atenção ajuda nesta tarefa?

Inverter uma sequência exige um **alinhamento posicional preciso**: o 1.º token de saída deve corresponder ao último token de entrada, e assim por diante. Com atenção, o decoder aprende exatamente esse mapa — o peso de atenção no passo `t` deve ser alto para a posição `(n - t)` da entrada. Sem atenção, o encoder precisa codificar toda essa informação posicional em um único vetor de tamanho fixo, o que é muito mais difícil.